In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
from tensorflow.python.framework.ops import EagerTensor
from matplotlib.pyplot import imshow

In [ ]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d agrigorev/clothing-dataset-full

clothing-dataset-full.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
from zipfile import ZipFile
file_name = 'clothing-dataset-full.zip' #the file is your dataset exact name
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [ ]:
# Import the dataset 
labels = pd.read_csv('/content/images.csv')

In [ ]:
labels

,image,sender_id,label,kids
0,4285fab0-751a-4b74-8e9b-43af05deee22,124,Not sure,False
1,ea7b6656-3f84-4eb3-9099-23e623fc1018,148,T-Shirt,False
2,00627a3f-0477-401c-95eb-92642cbe078d,94,Not sure,False
3,ea2ffd4d-9b25-4ca8-9dc2-bd27f1cc59fa,43,T-Shirt,False
4,3b86d877-2b9e-4c8b-a6a2-1d87513309d0,189,Shoes,False
...,...,...,...,...
5398,dfd4079d-967b-4b3e-8574-fbac11b58103,204,Shorts,False
5399,befa14be-8140-4faf-8061-1039947e329d,204,Body,True
5400,5379356a-40ee-4890-b416-2336a7d84061,310,Shorts,False
5401,65507fb8-3456-4c15-b53e-d1b03bf71a59,204,Shoes,False


In [ ]:
# Classes and their counts within the dataset
labels['label'].value_counts()

T-Shirt       1011
Longsleeve     699
Pants          692
Shoes          431
Shirt          378
Dress          357
Outwear        312
Shorts         308
Not sure       228
Hat            171
Skirt          155
Polo           120
Undershirt     118
Blazer         109
Hoodie         100
Body            69
Other           67
Top             43
Blouse          23
Skip            12
Name: label, dtype: int64

In [ ]:
labels['label'].unique()

array(['Not sure', 'T-Shirt', 'Shoes', 'Shorts', 'Shirt', 'Pants',
       'Skirt', 'Other', 'Top', 'Outwear', 'Dress', 'Body', 'Longsleeve',
       'Undershirt', 'Hat', 'Polo', 'Blouse', 'Hoodie', 'Skip', 'Blazer'],
      dtype=object)

In [ ]:
labels.loc[labels['label']=='Not sure','label'] = 'Not_sure'

In [ ]:
labels['image'] = labels['image'] + '.jpg'



In [ ]:
labels['label_cat'] = labels['label'] + ' ' + labels['kids'].astype(str)


In [ ]:
label_df = labels[['image', 'label_cat']]

In [ ]:
label_df

,image,label_cat
0,4285fab0-751a-4b74-8e9b-43af05deee22.jpg,Not_sure False
1,ea7b6656-3f84-4eb3-9099-23e623fc1018.jpg,T-Shirt False
2,00627a3f-0477-401c-95eb-92642cbe078d.jpg,Not_sure False
3,ea2ffd4d-9b25-4ca8-9dc2-bd27f1cc59fa.jpg,T-Shirt False
4,3b86d877-2b9e-4c8b-a6a2-1d87513309d0.jpg,Shoes False
...,...,...
5398,dfd4079d-967b-4b3e-8574-fbac11b58103.jpg,Shorts False
5399,befa14be-8140-4faf-8061-1039947e329d.jpg,Body True
5400,5379356a-40ee-4890-b416-2336a7d84061.jpg,Shorts False
5401,65507fb8-3456-4c15-b53e-d1b03bf71a59.jpg,Shoes False


In [ ]:
path = '/content'
def get_x(r): return path+'/images_original/'+r['image'] # create path to open images in the original folder
def get_y(r): return r['label_cat'].split(' ') # split the labels using space as a delimitter

In [ ]:
from tensorflow.keras.applications import resnet50

# Init the VGG model 
res_conv = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(128,128, 3))

In [ ]:
# Freeze all the layers
for layer in res_conv.layers[:]:
    layer.trainable = False

# Check the trainable status of the individual layers
for layer in res_conv.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7f25b71bb650> False
<keras.layers.convolutional.ZeroPadding2D object at 0x7f25b7482fd0> False
<keras.layers.convolutional.Conv2D object at 0x7f25b7490350> False
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f25b747c4d0> False
<keras.layers.core.activation.Activation object at 0x7f25b7482750> False
<keras.layers.convolutional.ZeroPadding2D object at 0x7f25b70ed950> False
<keras.layers.pooling.MaxPooling2D object at 0x7f25b746e3d0> False
<keras.layers.convolutional.Conv2D object at 0x7f25b74a7810> False
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f25b7721910> False
<keras.layers.core.activation.Activation object at 0x7f25b7721d50> False
<keras.layers.convolutional.Conv2D object at 0x7f25b7720b50> False
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f25b73f0750> False
<keras.layers.core.activation.Activation object at 0x7f25b73f1190> False

In [ ]:
# Create the model
model = Sequential()

# Add the resnet convolutional base model
model.add(res_conv)

# Add new layers
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(3, activation='softmax'))

# Show a summary of the model. Check the number of trainable parameters
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 flatten_5 (Flatten)         (None, 32768)             0         
                                                                 
 dense_8 (Dense)             (None, 1024)              33555456  
                                                                 
 dense_9 (Dense)             (None, 3)                 3075      
                                                                 
Total params: 57,146,243
Trainable params: 33,558,531
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
# I implemented transfer learning, but

In [ ]:
# I just cant find a way to fit the data into the model, m getting errors, looked over the internet, didnt find anything .

In [ ]:
# 